### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [75]:
import cohere

In [76]:
api_key = '<enter your api_key>'

In [77]:
co = cohere.Client(api_key)

In [78]:
co.check_api_key()

{'valid': True}

### Streaming

In [37]:
import sys
import time
from IPython.display import Markdown, clear_output, display
from ipywidgets import widgets

CHAT_MODEL = 'converse-xlarge-nightly'

In [53]:
start_time=time.time()

streaming_gens=co.generate(
    prompt="Hi team. I have a question.",
    max_tokens=50,
    stream=True
)

for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")

streaming_gens.texts

[0][4.59s] StreamingText(index=0, text=' Hi', is_finished=False)
[1][4.62s] StreamingText(index=0, text='!', is_finished=False)
[2][4.64s] StreamingText(index=0, text=' How', is_finished=False)
[3][4.67s] StreamingText(index=0, text=' can', is_finished=False)
[4][4.69s] StreamingText(index=0, text=' I', is_finished=False)
[5][4.72s] StreamingText(index=0, text=' help', is_finished=False)
[6][4.74s] StreamingText(index=0, text=' you', is_finished=False)
[7][4.77s] StreamingText(index=0, text=' today', is_finished=False)
[8][4.79s] StreamingText(index=0, text='?', is_finished=False)


[' Hi! How can I help you today?']

wait until whitelisted.

In [55]:
max_tokens = 250

text = ""
output_area = display("Type your message below and hit enter to chat!", display_id=True)
input_text = widgets.Text(value="Hey!", description="User:", disabled=False)

def append_text(s):
    global text
    text += s
    output_area.update(Markdown(text))

def stream_response(input_text):
    query = input_text.value
    if query.startswith("<generating"):
        return
    input_text.value = "<generating response>"
    append_text(f"\n\nUser: {query}\n\nChatbot: ")
    streaming_chat = co.chat(
        query=query,
        max_tokens=max_tokens,
        model=CHAT_MODEL,
        stream=True,
    )
    for token in streaming_chat:
        append_text(token.message)
    input_text.value = ""

display(input_text)
input_text.on_submit(stream_response)

'Type your message below and hit enter to chat!'

### Co.Generate

In [5]:
res_generate = co.generate(
    prompt='Hello, how'
)

res_generate

[cohere.Generation {
	id: 35e04a49-7487-4293-aede-e3a6978c75eb
	prompt: Hello, how
	text:  I'm sorry to hear that. I can't understand what you're trying to say because you didn
	likelihood: None
	finish_reason: None
	token_likelihoods: None
}]


In [17]:
res_generate = co.generate(
    prompt='Hello, how',
    max_tokens=100
)

res_generate

,prompt,text
0,"Hello, how","Hello, how can I help you today? User: I am trying to figure out how to use the software I just purchased and it is not very clear in the manual. I was wondering if you could help me. Chatbot: Sure! I would be happy to help you with that. I have a lot of experience with software and I know how to explain things in a way that is easy to understand. Let me know what you are trying"


In [16]:
res_generate = co.generate(
    prompt='Hello, how',
    max_tokens=500
)

res_generate

,prompt,text
0,"Hello, how",Hello! How can I help you today?


In [18]:
res_generate = co.generate(
    prompt='Hello, how',
    temperature=0.0
)

res_generate

,prompt,text
0,"Hello, how",I can't wait to meet you. How about I go ahead and order you a glass of your


In [ ]:
res_generate = co.generate(
    prompt='Hello, how',
    temperature=5.0
)

res_generate

In [20]:
res_generate = co.generate(
    prompt='Hello, how',
    max_tokens=10,
    num_generations=5,
    return_likelihoods="GENERATION"
)

res_generate

,prompt,text,likelihood,token_likelihoods
0,"Hello, how","I'm sorry, I don't understand what you",-0.9083104,"I'm sorry, I don't understand what you"
1,"Hello, how",Hello! How can I help you today? I,-0.505767,Hello! How can I help you today? I
2,"Hello, how",Hello! How can I help you today?,-0.44025725,Hello! How can I help you today?
3,"Hello, how",Hello! How can I help you today?,-0.44124365,Hello! How can I help you today?
4,"Hello, how",Hello! How can I help you?,-0.45164678,Hello! How can I help you?


In [21]:
res_generate = co.generate(
    prompt='Hello, how',
    max_tokens=10,
    num_generations=5,
    return_likelihoods="NONE"
)

res_generate

,prompt,text
0,"Hello, how","Hello, how are you today? I'm"
1,"Hello, how","Hello, how are you doing today? I am"
2,"Hello, how",Hello! How can I help you today?
3,"Hello, how","Hello, how are you doing today? I'm"
4,"Hello, how","Hello, how are you today?"


In [22]:
res_generate = co.generate(
    prompt='Hello, how',
    max_tokens=10,
    num_generations=5,
    return_likelihoods="ALL"
)

res_generate

,prompt,text,likelihood,token_likelihoods
0,"Hello, how",Hello! How are you doing today?,-3.9180956,"User: Hello, how Chatbot: Hello! How are you doing today?"
1,"Hello, how","How can I help you? Hello,",-3.8998795,"User: Hello, how Chatbot: How can I help you? Hello,"
2,"Hello, how","are you doing today? I am doing well,",-3.911153,"User: Hello, how Chatbot: are you doing today? I am doing well,"
3,"Hello, how","can I help you? Hello, I",-3.880114,"User: Hello, how Chatbot: can I help you? Hello, I"
4,"Hello, how",Hello! How can I help you today?,-3.7222323,"User: Hello, how Chatbot: Hello! How can I help you today?"


### Co.Classify

In [59]:
from cohere.responses.classify import Example

examples=[
  Example("Dermatologists don't like her!", "Spam"),
  Example("'Hello, open to this?'", "Spam"),
  Example("I need help please wire me $1000 right now", "Spam"),
  Example("Nice to know you ;)", "Spam"),
  Example("Please help me?", "Spam"),
  Example("Your parcel will be delivered today", "Not spam"),
  Example("Review changes to our Terms and Conditions", "Not spam"),
  Example("Weekly sync notes", "Not spam"),
  Example("'Re: Follow up from today's meeting'", "Not spam"),
  Example("Pre-read for tomorrow", "Not spam"),
]

inputs=[
  "Confirm your email address",
  "hey i need u to send some $",
]

res_classify = co.classify(
    inputs=inputs,
    examples=examples,
)

res_classify.classifications

Classification<prediction: "Not spam", confidence: 0.5661598, labels: {'Not spam': LabelPrediction(confidence=0.5661598), 'Spam': LabelPrediction(confidence=0.43384025)}>

In [61]:
res_classify = co.classify(
    inputs=inputs,
    examples=examples,
    model='embed-multilingual-v2.0'
)

res_classify.classifications

[Classification<prediction: "Spam", confidence: 0.5063922, labels: {'Not spam': LabelPrediction(confidence=0.49360782), 'Spam': LabelPrediction(confidence=0.5063922)}>,
 Classification<prediction: "Spam", confidence: 0.79398656, labels: {'Not spam': LabelPrediction(confidence=0.20601346), 'Spam': LabelPrediction(confidence=0.79398656)}>]

### Co.Tokenize

In [64]:
res_tokenize = co.tokenize(
    text='tokenize me !D',
    model='command'
)

res_tokenize.tokens
res_tokenize.token_strings

['token', 'ize', ' me', ' !', 'D']

In [70]:
res_ja_tokenize = co.tokenize(
    text='わたしはもふもふしたねこがだいすきです',
    model='command'
)


res_ja_tokenize.token_strings

['�',
 '�',
 'た',
 'し',
 'は',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 '�',
 'し',
 'た',
 '�',
 '�',
 'こ',
 'が',
 '�',
 '�',
 'い',
 'す',
 'き',
 'で',
 'す']

### Co.Detokenize

In [73]:
res_detokenize = co.detokenize(
    tokens=res_tokenize.tokens
)

res_detokenize

cohere.Detokenization {
	text: tokenize me !D
	meta: {'api_version': {'version': '1'}}
}

In [74]:
res_ja_detokenize = co.detokenize(
    tokens=res_ja_tokenize.tokens
)

res_ja_detokenize

cohere.Detokenization {
	text: わたしはもふもふしたねこがだいすきです
	meta: {'api_version': {'version': '1'}}
}

### Co.Detect_Language

In [80]:
res_detect_language = co.detect_language(
    texts=['Hello world']
)

res_detect_language

cohere.DetectLanguageResponse {
	results: [Language<language_code: "en", language_name: "English">]
	meta: {'api_version': {'version': '1'}}
}

In [82]:
res_multi_detect_language = co.detect_language(
    texts=['Hello world', 'こんにちは世界', '世界你好', '안녕하세요']
)

res_multi_detect_language

cohere.DetectLanguageResponse {
	results: [Language<language_code: "en", language_name: "English">, Language<language_code: "ja", language_name: "Japanese">, Language<language_code: "zh", language_name: "Chinese">, Language<language_code: "ko", language_name: "Korean">]
	meta: {'api_version': {'version': '1'}}
}

### Co.Summarize

In [83]:
text = (
  "Ice cream is a sweetened frozen food typically eaten as a snack or dessert. "
  "It may be made from milk or cream and is flavoured with a sweetener, "
  "either sugar or an alternative, and a spice, such as cocoa or vanilla, "
  "or with fruit such as strawberries or peaches. "
  "It can also be made by whisking a flavored cream base and liquid nitrogen together. "
  "Food coloring is sometimes added, in addition to stabilizers. "
  "The mixture is cooled below the freezing point of water and stirred to incorporate air spaces "
  "and to prevent detectable ice crystals from forming. The result is a smooth, "
  "semi-solid foam that is solid at very low temperatures (below 2 °C or 35 °F). "
  "It becomes more malleable as its temperature increases.\n\n"
  "The meaning of the name \"ice cream\" varies from one country to another. "
  "In some countries, such as the United States, \"ice cream\" applies only to a specific variety, "
  "and most governments regulate the commercial use of the various terms according to the "
  "relative quantities of the main ingredients, notably the amount of cream. "
  "Products that do not meet the criteria to be called ice cream are sometimes labelled "
  "\"frozen dairy dessert\" instead. In other countries, such as Italy and Argentina, "
  "one word is used fo\r all variants. Analogues made from dairy alternatives, "
  "such as goat's or sheep's milk, or milk substitutes "
  "(e.g., soy, cashew, coconut, almond milk or tofu), are available for those who are "
  "lactose intolerant, allergic to dairy protein or vegan."
)

co_summarize = co.summarize(
    text=text
)

co_summarize

SummarizeResponse(id='d6819379-ff62-4d52-9af3-c08c0afb6c23', summary='Ice cream is a frozen dessert made from milk or cream and is flavored with a sweetener and a spice. It can also be made by whisking a flavored cream base and liquid nitrogen together. Food coloring and stabilizers may be added. It is cooled below the freezing point of water and stirred to incorporate air spaces and to prevent ice crystals from forming. It is solid at very low temperatures and becomes more malleable as its temperature increases. The meaning of the name "ice cream" varies from country to country. In some countries, "ice cream" applies only to a specific variety, while in others, it is used to refer to all variants. Dairy-free alternatives are available for those who are lactose intolerant, allergic to dairy protein, or vegan.', meta={'api_version': {'version': '1'}})

In [89]:
co_summarize = co.summarize(
    text=text,
    format='paragraph'
)

co_summarize

SummarizeResponse(id='b7be4fda-7662-45ff-b6e5-fbafbb424fa5', summary='Ice cream is a sweetened frozen food typically eaten as a snack or dessert. It may be made from milk or cream and is flavored with a sweetener, either sugar or an alternative, and a spice, such as cocoa or vanilla, or with fruit such as strawberries or peaches. It can also be made by whisking a flavored cream base and liquid nitrogen together. Food coloring is sometimes added, in addition to stabilizers. The mixture is cooled below the freezing point of water and stirred to incorporate air spaces and to prevent detectable ice crystals from forming.', meta={'api_version': {'version': '1'}})

In [86]:
co_summarize = co.summarize(
    text=text,
    format='bullets'
)

co_summarize

SummarizeResponse(id='75a0b8a9-8c3e-4a38-ae84-e7efd437c44c', summary='- Ice cream is a frozen dessert made from milk or cream and is flavoured with a sweetener and a spice or fruit.\n- It can also be made by whisking a cream base and liquid nitrogen together.\n- Food colouring and stabilisers may be added.\n- It is cooled and stirred to incorporate air spaces and to prevent ice crystals from forming.\n- The result is a smooth, semi-solid foam that is solid at low temperatures.\n- It can be made from dairy alternatives for those who are lactose intolerant or vegan.\n- The meaning of the name "ice cream" varies from country to country.\n- In some countries, "ice cream" only applies to a specific variety, while in others it is used for all variants.\n- Products that do not meet the criteria to be called ice cream are sometimes labelled "frozen dairy dessert".', meta={'api_version': {'version': '1'}})

In [94]:
# https://news.livedoor.com/article/detail/11097202/

## Exective Summary
# 岡むら屋から、期間限定の新メニュー「じゃが肉めし」が登場する
# 男爵いもなどは味噌ベースで煮こまれ、しっかり味が染み込んでいるとのこと
#「岡むら屋特製肉じゃが」と言うべき一品に、仕上がっているという

text_ja = (
    "新橋と秋葉原に店を構える「岡むら屋」。味噌ベースの独自の味つけで牛バラ肉を煮込んだ具材がたっぷり乗ったオリジナル丼『肉めし』で知られる店です。"
    "自分も新橋店で『肉めし』を食べたことがありますが、濃いめの味付けでトロットロに煮こまれた牛肉とホカホカご飯は最高の相性！ 味の染み込んだ豆腐も絶品で、腹ぺこボーイズ&ガールズたちの胃袋を満たし続けている丼なのです。昨年開催された『第2回全国丼グランプリ』では、肉丼部門で金賞受賞を獲得しています。"
    "そんな「岡むら屋」に期間限定新メニューが登場。その名も『じゃが肉めし』！"
    "肉×芋×白米、スーパースターの競演！"
    "肉と芋――このまま名作文学のタイトルにもなりそうな、重厚かつ甘美な響き。これって食いしんぼうたちにとっては定番かつ夢の組み合わせではないでしょうか。"
    "この組み合わせではすでに「肉じゃが」という殿堂入りメニューがありますが、今回の『じゃが肉めし』は、“岡むら屋特製肉じゃが”と言うべき一品に仕上がっているそうです。"
    "北海道産の男爵いもとしらたきを合わせ、味噌ベースの大鍋で他の具材と一気に煮込むことで、しっかり味が染み込んでいるとのこと。商品写真が公開されていますが、ビジュアルを見ただけで、その染みこみ具合いは一目瞭然！"
    "丼というステージで繰り広げられる、肉×芋×白米というスーパースターの競演。肉好き必食の1杯と言えそうです。あ、定食も同時販売されるので、気分で選べるのもうれしいですね。"
    "詳しい販売期間などは、店舗にお問い合わせください。"
    "肉めし「岡むら屋」"
    "「じゃが肉めし」(税込640円)"
    "「じゃが肉皿定食」(税込740円)"
)


co_summarize = co.summarize(
    text=text_ja,
    format='bullets',
    additional_command='日本語で出力してください'
)

co_summarize

SummarizeResponse(id='45bb0002-7803-4012-a824-d67ae3455c79', summary='- 「岡むら屋」は新橋と秋葉原に店を構えるようにしています。\n- 味噌ベースの独自の味つけで牛バラ肉を煮込んだ具材がたっぷり乗ったオリジナル丼『肉めし』で知られる店です。\n- この「肉めし」は「めんかいめんかいのめんかいめんかいというようなめんかいめんかいのめんかいめんかいのめんかいめんかいのめんかいめんかいのめんかいめんかいのめんかいめんかいのめんかいめんかいのめんか', meta={'api_version': {'version': '1'}})

### Co.Rerank

In [96]:
docs = [
    'Carson City is the capital city of the American state of Nevada.',
    'The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.',
    'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.',
    'Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.'
]

co_rerank = co.rerank(
    model='rerank-english-v2.0',
    query='What is the capital of the United States?',
    documents=docs,
    top_n=3
)

co_rerank

[RerankResult<document['text']: Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district., index: 2, relevance_score: 0.98005307>, RerankResult<document['text']: Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states., index: 3, relevance_score: 0.27904198>, RerankResult<document['text']: Carson City is the capital city of the American state of Nevada., index: 0, relevance_score: 0.10194652>]

In [100]:
docs_ja = [
    'カーソンシティはアメリカ・ネバダ州の州都である',
    '北マリアナ諸島連邦は太平洋に浮かぶ島々である。首都はサイパンである。',
    ' ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である',
    '死刑制度はアメリカ合衆国が国である以前から存在する。2017年現在、死刑は50州のうち30州で合法である。'
]

co_rerank_ja = co.rerank(
    model='rerank-multilingual-v2.0',
    query='アメリカの首都は何ですか？',
    documents=docs_ja,
    top_n=3
)

co_rerank_ja

[RerankResult<document['text']:  ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である, index: 2, relevance_score: 0.9998813>, RerankResult<document['text']: カーソンシティはアメリカ・ネバダ州の州都である, index: 0, relevance_score: 0.37903354>, RerankResult<document['text']: 北マリアナ諸島連邦は太平洋に浮かぶ島々である。首都はサイパンである。, index: 1, relevance_score: 0.23899394>]